# Lab 1. Hand-crafted graph features

In [150]:
pip install grakel

In [151]:
import networkx as nx
import numpy as np
import random
from collections import defaultdict

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

from grakel import GraphKernel

## 1. Подготовка данных

Сгенерируем набор данных для бинарной классификации графов.
Будем использовать графы-пути и графы-циклы

In [152]:
def create_dataset(size = 100, nodes = 30):

    graph_list = [nx.cycle_graph(nodes) for i in range(10, size//2 + 10)]
    graph_list.extend([nx.path_graph(nodes) for i in range(size//2 + 10, size + 10)])
    y = [0 if i < (size//2 + 10) else 1 for i in range(size)]

    return graph_list, y

## 2. Реализация Shortest Path Kernel

Функция для вычисления ядра кратчайших путей на основе векторов, которые содержат количество кратчайших путей различной длины для каждого графа.

In [172]:
def shortest_path_kernel(graphs_train, graphs_test, max_nodes=30):

    # Инициализируем массивы признаков для каждого графа в обучающем и тестовом наборах
    features_train = np.zeros((len(graphs_train), max_nodes * (max_nodes - 1)))
    features_test = np.zeros((len(graphs_test), max_nodes * (max_nodes - 1)))

    # Вычисляем признаки для обучающего набора графов
    for index, graph in enumerate(graphs_train):
        position = 0  # Позиция в векторе признаков
        for start_node in range(graph.number_of_nodes() - 1):
            for end_node in range(start_node + 1, graph.number_of_nodes()):
                # Рассчитываем кратчайшее расстояние между парами узлов
                features_train[index][position] = nx.shortest_path_length(graph, start_node, end_node)
                position += 1

    # Вычисляем признаки для тестового набора графов
    for index, graph in enumerate(graphs_test):
        position = 0  # Позиция в векторе признаков
        for start_node in range(graph.number_of_nodes() - 1):
            for end_node in range(start_node + 1, graph.number_of_nodes()):
                # Рассчитываем кратчайшее расстояние между парами узлов
                features_test[index][position] = nx.shortest_path_length(graph, start_node, end_node)
                position += 1

    # Вычисляем матрицы ядра через скалярное произведение признаков
    kernel_train = np.dot(features_train, features_train.T)
    kernel_test = np.dot(features_test, features_train.T)

    return kernel_train, kernel_test

## 3. Обучение модели SVC

In [193]:
graph_list, y = create_dataset(1000)
train_graphs, test_graphs, y_train, y_test = train_test_split(graph_list, y, test_size=0.1, stratify=y)

K_train_gk, K_test_gk = shortest_path_kernel(train_graphs, test_graphs)

svc = SVC(kernel='precomputed', random_state = 42)

param_grid = {'C': [1, 10, 100], 'tol': [0.01, 0.001, 0.0001]}
grid_search = GridSearchCV(svc, param_grid)
grid_search.fit(K_train_gk, y_train)

grid_search.best_params_

{'C': 1, 'tol': 0.01}

In [194]:
best_model = SVC(C = 1, kernel ='precomputed', tol = 0.01, random_state = 42)

best_model.fit(K_train_gk, y_train)
y_pred = best_model.predict(K_test_gk)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        51
           1       0.96      1.00      0.98        49

    accuracy                           0.98       100
   macro avg       0.98      0.98      0.98       100
weighted avg       0.98      0.98      0.98       100



## 4. Реализация Weisfeiler-Lehman Kernel

Теперь реализуем ядро Вайсфейлера-Лемана (WL Kernel), который сравнивает графы на основе их структурных инвариантов, построенных с использованием метода хеширования меток узлов.

In [156]:
def weisfeiler_lehman_kernel(train_graphs, test_graphs, iterations=30):

    # Инициализация векторов признаков для тренировочных и тестовых графов
    phi_train = []
    phi_test = []

    # Проход по каждому графу из обучающего набора
    for i, graph in enumerate(train_graphs):
        phi_train_dict = {1: graph.number_of_nodes()}  # Словарь признаков (ключ - метка, значение - количество узлов с данной меткой)
        hash_map = {'1': '1'}  # Хэш-таблица для хранения уникальных меток
        node_labels = {node: 1 for node in graph.nodes()}  # Начальная метка для всех узлов

        # Цикл по количеству итераций WL
        for it in range(iterations):
            # Добавляем дополнительные ячейки в словарь признаков для данной итерации
            for extra in range(it * 10):
                phi_train_dict[extra + 2] = 0

            # Временный словарь для обновленных меток узлов на текущей итерации
            temp_hash = {}

            # Обновляем метки узлов с учетом меток их соседей
            for node in graph.nodes():
                # Формируем строку меток текущего узла и его соседей
                neighbor_labels = ''.join(sorted(str(node_labels[neighbor]) for neighbor in graph.neighbors(node)))
                temp_label = f"{node_labels[node]},{neighbor_labels}"
                temp_hash[node] = temp_label

            # Обновляем глобальные метки, если встретились новые комбинации
            for node in sorted(temp_hash, key=lambda k: temp_hash[k]):
                if temp_hash[node] not in hash_map.values():
                    max_key = len(hash_map) + 1
                    hash_map[max_key] = temp_hash[node]

            # Обновляем метки узлов в словаре node_labels
            for node, label in temp_hash.items():
                new_label = next(k for k, v in hash_map.items() if v == label)
                node_labels[node] = new_label
                phi_train_dict[new_label] = phi_train_dict.get(new_label, 0) + 1

        # Сортировка признаков и преобразование в вектор
        sorted_features = sorted(phi_train_dict.items())
        feature_vector = np.array([count for _, count in sorted_features])
        phi_train.append(feature_vector)

    # Аналогичные вычисления для тестовых графов
    for i, graph in enumerate(test_graphs):
        phi_test_dict = {1: graph.number_of_nodes()}
        hash_map = {'1': '1'}
        node_labels = {node: 1 for node in graph.nodes()}

        for it in range(iterations):
            for extra in range(it * 10):
                phi_test_dict[extra + 2] = 0

            temp_hash = {}

            for node in graph.nodes():
                neighbor_labels = ''.join(sorted(str(node_labels[neighbor]) for neighbor in graph.neighbors(node)))
                temp_label = f"{node_labels[node]},{neighbor_labels}"
                temp_hash[node] = temp_label

            for node in sorted(temp_hash, key=lambda k: temp_hash[k]):
                if temp_hash[node] not in hash_map.values():
                    max_key = len(hash_map) + 1
                    hash_map[max_key] = temp_hash[node]

            for node, label in temp_hash.items():
                new_label = next(k for k, v in hash_map.items() if v == label)
                node_labels[node] = new_label
                phi_test_dict[new_label] = phi_test_dict.get(new_label, 0) + 1

        sorted_features = sorted(phi_test_dict.items())
        feature_vector = np.array([count for _, count in sorted_features])
        phi_test.append(feature_vector)

    # Вычисление матриц ядровых функций
    K_train = np.dot(phi_train, np.array(phi_train).T)
    K_test = np.dot(phi_test, np.array(phi_train).T)

    return K_train, K_test

In [197]:
graph_list, y = create_dataset(1000)

train_graphs, test_graphs, y_train, y_test = train_test_split(graph_list, y, test_size=0.1, stratify=y)
K_train_gk2, K_test_gk2 = weisfeiler_lehman_kernel(train_graphs, test_graphs, 10)

svc2 = SVC(C = 1, kernel ='precomputed', tol = 0.01, random_state = 42)

param_grid = {'C': [1, 10, 100], 'tol': [0.01, 0.001, 0.0001]}
grid_search = GridSearchCV(svc2, param_grid)
grid_search.fit(K_train_gk, y_train)

grid_search.best_params_

{'C': 1, 'tol': 0.01}

In [199]:
best_model2 = SVC(C = 1, kernel ='precomputed', tol = 0.01, random_state = 42)

best_model2.fit(K_train_gk2, y_train)
y_pred2 = best_model2.predict(K_test_gk2)

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        51
           1       0.94      1.00      0.97        49

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100



1. `Weisfeiler-Lehman kernel` выполняется значительно быстрее, так как он работает с метками узлов и использует итеративное обновление этих меток для захвата структурной информации. В отличие от этого, `shortest_path_kernel()` требует вычислений кратчайших путей для всех пар узлов, что значительно увеличивает время работы для больших графов.

3. `shortest_path_kernel()` более точный, поскольку сохраняет более детальную информацию о графе, так как учитывает расстояния между всеми узлами. В `Weisfeiler-Lehman kernel` информация о графе представлена более абстрактно через метки узлов, что снижает точность по сравнению с кратчайшими путями.

В целом, `Weisfeiler-Lehman kernel` является хорошим выбором для быстрого анализа крупных графов с умеренной детализацией, тогда как `shortest_path_kernel()` может быть полезен, если требуется максимальная точность и графы небольшие.